# Profili di corsa per Spa

In [43]:
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import numpy as np
import os
from PIL import Image
import glob
%matplotlib inline

In [44]:
# Select map
MAP_NAME = "Spa_map"

In [45]:
csv_files = glob.glob(f'/home/weins/sim_ws/csv/*spa*', recursive=False)
for i, x in enumerate(csv_files):
    print(f"[{i}]: {x}")

[0]: /home/weins/sim_ws/csv/pp_spa_controls_out.csv
[1]: /home/weins/sim_ws/csv/mpc_spa_hp_final_out.csv
[2]: /home/weins/sim_ws/csv/mpc_spa_hp_controls_out.csv
[3]: /home/weins/sim_ws/csv/mpc_spa_fast_final_out.csv
[4]: /home/weins/sim_ws/csv/mpc_spa_safe_controls_out.csv
[5]: /home/weins/sim_ws/csv/spa_pp_final_out.csv
[6]: /home/weins/sim_ws/csv/mpc_spa_safe_final_out.csv
[7]: /home/weins/sim_ws/csv/traj_race_spa_v2.csv
[8]: /home/weins/sim_ws/csv/traj_race_spa.csv
[9]: /home/weins/sim_ws/csv/spa_mpc_final_out.csv
[10]: /home/weins/sim_ws/csv/trajrace_pp_spa.csv
[11]: /home/weins/sim_ws/csv/mpc_spa_fast_controls_out.csv
[12]: /home/weins/sim_ws/csv/pp_spa_final_out.csv


In [46]:
# Select file index for 'mintime', 'mincurv', 'shortest_path', 'centerline'
ref, pp, hp, fast, safe = 7, 12, 1, 3, 6

In [47]:
ref_df = pd.read_csv(csv_files[ref], names=['x', 'y', 'theta', 'speed'], sep=',')
pp_df = pd.read_csv(csv_files[pp], sep=',')
hp_df = pd.read_csv(csv_files[hp], sep=',')
fast_df = pd.read_csv(csv_files[fast], sep=',')
safe_df = pd.read_csv(csv_files[safe], sep=',')

data = [
    (ref_df, '#e7298a', 'Raceline'),#(ref_df, 'red', 'Raceline'),
    (pp_df, '#1b9e77', 'Pure Pursuit'), #(pp_df, 'sienna', 'Pure Pursuit'),
    (hp_df, '#e6ab02', 'MPC High Performance'), #(hp_df, 'orange', 'MPC High Performance'), 
    (fast_df, '#d95f02', 'MPC Fast'), #(fast_df, 'blue', 'MPC Fast'),  # lightblue
    (safe_df, '#7570b3', 'MPC Safe') #(safe_df, 'green', 'MPC Safe')
]

In [48]:
ref_df

,x,y,theta,speed
0,11.175317,-66.022806,-0.377448,10.267294
1,11.230135,-65.883709,-0.373357,10.251275
2,11.284422,-65.744271,-0.369176,10.235257
3,11.338161,-65.604497,-0.364904,10.219238
4,11.391333,-65.464394,-0.360540,10.203220
...,...,...,...,...
3635,10.949108,-66.579835,-0.394222,10.436610
3636,11.006471,-66.441106,-0.389941,10.436610
3637,11.063287,-66.302025,-0.385718,10.436610
3638,11.119566,-66.162592,-0.381553,10.436610


In [49]:
# Overlay the racetrack on the map
# load map yaml
if os.path.exists(f"/home/weins/sim_ws/src/f1tenth_gym_ros/maps/{MAP_NAME}.png"):
    map_img_path = f"/home/weins/sim_ws/src/f1tenth_gym_ros/maps/{MAP_NAME}.png"
else:
    raise Exception("Map not found!")

map_yaml_path = f"/home/weins/sim_ws/src/f1tenth_gym_ros/maps/{MAP_NAME}.yaml"
with open(map_yaml_path, 'r') as yaml_stream:
    try:
        map_metadata = yaml.safe_load(yaml_stream)
        map_resolution = map_metadata['resolution']
        origin = map_metadata['origin']
    except yaml.YAMLError as ex:
        print(ex)

# calculate map parameters
orig_x = origin[0]
orig_y = origin[1]
# ??? Should be 0>
orig_s = np.sin(origin[2])
orig_c = np.cos(origin[2])

def transf(raw):
    # get the distance transform
    transformed_data = raw.copy()
    transformed_data -= np.array([orig_x, orig_y] + [0]*(len(raw.columns)-2))
    transformed_data.iloc[:, :2] /= map_resolution
    return transformed_data

raw_map_img = np.array(Image.open(map_img_path).transpose(Image.FLIP_TOP_BOTTOM))
raw_map_img = raw_map_img.astype(np.float64)

transf_data = []
for raw in data:
    transf_data.append( (transf(raw[0]), raw[1], raw[2]) )

In [50]:
# Image resolution
size = 12

In [51]:
%matplotlib qt
plt.figure(figsize=(size, size))
for d, c, l in transf_data:
    plt.plot(d.loc[:, 'x'].to_numpy(), d.loc[:, 'y'].to_numpy(), c = c, label=l) #, c=transformed_data.loc[:, i], marker=".", s=3)
    #plt.scatter(d.loc[:, 'x'].to_numpy(), d.loc[:, 'y'].to_numpy(), c = c, s=2) #, c=transformed_data.loc[:, i], marker=".", s=3)
#plt.scatter(transformed_data.loc[0, 'x'], transformed_data.loc[0, 'y'], c='red', marker=(3,0,transformed_data.loc[0, 'a']*180/3.1415), s=200)

#plt.xlim(transf_data[0].iloc[:, 0].min() - 100, transf_data[0].iloc[:, 0].max() + 100)
#plt.ylim(transf_data[0].iloc[:, 1].min() - 100, transf_data[0].iloc[:, 1].max() + 100)
plt.imshow(raw_map_img, cmap='gray', origin='lower')
plt.legend(fontsize=16)
plt.axis('off')

(-0.5, 1999.5, -0.5, 1999.5)

kf.kio.widgets.kdirmodel: No node found for item that was just removed: QUrl("file:///home/weins/Documents/Grafici/MPC and PP/Lap Time Comparisons Monza.png")
kf.kio.widgets.kdirmodel: No node found for item that was just removed: QUrl("file:///home/weins/Documents/Grafici/MPC and PP/Lap Time Comparisons Monza.png")


## Monza

In [35]:
# Select map
MAP_NAME = "Monza_map"

In [36]:
csv_files = glob.glob(f'/home/weins/sim_ws/csv/*monza*', recursive=False)
for i, x in enumerate(csv_files):
    print(f"[{i}]: {x}")

[0]: /home/weins/sim_ws/csv/mpc_monza_safe_final_out.csv
[1]: /home/weins/sim_ws/csv/traj_race_monza_v2.csv
[2]: /home/weins/sim_ws/csv/monza_mpc_final_out.csv
[3]: /home/weins/sim_ws/csv/trajrace_pp_monza.csv
[4]: /home/weins/sim_ws/csv/mpc_monza_hp_final_out.csv
[5]: /home/weins/sim_ws/csv/monza_pp_final_out.csv
[6]: /home/weins/sim_ws/csv/mpc_monza_hp_controls_out.csv
[7]: /home/weins/sim_ws/csv/mpc_monza_fast_controls_out.csv
[8]: /home/weins/sim_ws/csv/pp_monza_controls_out.csv
[9]: /home/weins/sim_ws/csv/mpc_monza_fast_final_out.csv
[10]: /home/weins/sim_ws/csv/mpc_monza_safe_controls_out.csv
[11]: /home/weins/sim_ws/csv/pp_monza_final_out.csv
[12]: /home/weins/sim_ws/csv/traj_race_monza.csv


In [37]:
# Select file index for 'mintime', 'mincurv', 'shortest_path', 'centerline'
ref, pp, hp, fast, safe = 1, 11, 4, 9, 0

In [38]:
ref_df = pd.read_csv(csv_files[ref], names=['x', 'y', 'theta', 'speed'], sep=',')
pp_df = pd.read_csv(csv_files[pp], sep=',')
hp_df = pd.read_csv(csv_files[hp], sep=',')
fast_df = pd.read_csv(csv_files[fast], sep=',')
safe_df = pd.read_csv(csv_files[safe], sep=',')

data = [
    (ref_df, 'red', 'Raceline'),
    (pp_df, 'sienna', 'Pure Pursuit'),
    (hp_df, 'orange', 'MPC High Performance'), 
    (fast_df, 'blue', 'MPC Fast'),  # lightblue
    (safe_df, 'green', 'MPC Safe')
]

In [39]:
ref_df

,x,y,theta,speed
0,2.811887,33.892912,3.054638,12.547317
1,2.798863,33.743580,3.054567,12.562538
2,2.785829,33.594237,3.054500,12.577760
3,2.772783,33.444884,3.054437,12.592982
4,2.759728,33.295520,3.054377,12.608203
...,...,...,...,...
2939,2.863912,34.490694,3.054919,12.330430
2940,2.850922,34.341262,3.054851,12.330430
2941,2.837922,34.191821,3.054782,12.330430
2942,2.824910,34.042371,3.054711,12.330430


In [40]:
# Overlay the racetrack on the map
# load map yaml
if os.path.exists(f"/home/weins/sim_ws/src/f1tenth_gym_ros/maps/{MAP_NAME}.png"):
    map_img_path = f"/home/weins/sim_ws/src/f1tenth_gym_ros/maps/{MAP_NAME}.png"
else:
    raise Exception("Map not found!")

map_yaml_path = f"/home/weins/sim_ws/src/f1tenth_gym_ros/maps/{MAP_NAME}.yaml"
with open(map_yaml_path, 'r') as yaml_stream:
    try:
        map_metadata = yaml.safe_load(yaml_stream)
        map_resolution = map_metadata['resolution']
        origin = map_metadata['origin']
    except yaml.YAMLError as ex:
        print(ex)

# calculate map parameters
orig_x = origin[0]
orig_y = origin[1]
# ??? Should be 0>
orig_s = np.sin(origin[2])
orig_c = np.cos(origin[2])

def transf(raw):
    # get the distance transform
    transformed_data = raw.copy()
    transformed_data -= np.array([orig_x, orig_y] + [0]*(len(raw.columns)-2))
    transformed_data.iloc[:, :2] /= map_resolution
    return transformed_data

raw_map_img = np.array(Image.open(map_img_path).transpose(Image.FLIP_TOP_BOTTOM))
raw_map_img = raw_map_img.astype(np.float64)

transf_data = []
for raw in data:
    transf_data.append( (transf(raw[0]), raw[1], raw[2]) )

In [41]:
# Image resolution
size = 12

In [52]:
%matplotlib qt
plt.figure(figsize=(size, size))
for d, c, l in transf_data:
    plt.plot(d.loc[:, 'x'].to_numpy(), d.loc[:, 'y'].to_numpy(), c = c, label=l) #, c=transformed_data.loc[:, i], marker=".", s=3)
    #plt.scatter(d.loc[:, 'x'].to_numpy(), d.loc[:, 'y'].to_numpy(), c = c, s=2) #, c=transformed_data.loc[:, i], marker=".", s=3)
#plt.scatter(transformed_data.loc[0, 'x'], transformed_data.loc[0, 'y'], c='red', marker=(3,0,transformed_data.loc[0, 'a']*180/3.1415), s=200)

#plt.xlim(transf_data[0].iloc[:, 0].min() - 100, transf_data[0].iloc[:, 0].max() + 100)
#plt.ylim(transf_data[0].iloc[:, 1].min() - 100, transf_data[0].iloc[:, 1].max() + 100)
plt.imshow(raw_map_img, cmap='gray', origin='lower')
plt.legend(fontsize=16)
plt.axis('off')

(-0.5, 1999.5, -0.5, 1999.5)